夺冠队伍所在大洲分布

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
summary = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupsSummary.csv")
summary.createTempView("summary_tb")
summary.head()

Row(Year='1930', HostCountry='Uruguay', Winner='Uruguay', Second='Argentina', Third='USA', Fourth='Yugoslavia', GoalsScored='70', QualifiedTeams='13', MatchesPlayed='18', Attendance='590549', HostContinent='America', WinnerContinent='America')

In [2]:
continent_champion = spark.sql('''
SELECT continent,counts,ROUND ((counts/SUM(counts) OVER())*100,2) AS percentage FROM
(SELECT WinnerContinent AS continent ,COUNT(WinnerContinent) AS counts FROM summary_tb GROUP BY WinnerContinent) t ORDER BY counts DESC
''')

In [5]:
continent_champion.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "continent_champion") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .mode("append") \
    .save()

In [6]:
continent_champion.show()

+---------+------+----------+
|continent|counts|percentage|
+---------+------+----------+
|   Europe|    12|     57.14|
|  America|     9|     42.86|
+---------+------+----------+
